In [1]:
import pandas as pd
import numpy as np
import re,nltk
from numpy import array
import csv
import re, io

In [2]:
with open("orginaldata.txt", 'r', encoding="utf-8") as f:
    lines = f.readlines()
sentiment = []   
X = []
y = []

for line in lines:
    sentiment.append(line)
    
for el in sentiment:
    men = el.split(",")
    y.append(men[2])
    X.append(men[1])
    
print(len(X))
print(y[0:10])
print(y[29000:29010])
print(X[0:10])

3613
['1\n', '1\n', '1\n', '0\n', '0\n', '1\n', '1\n', '0\n', '0\n', '0\n']
[]
['ልክአደላችሁም ንብረታችንን የምርምር ስራ ከነፓተንቱ አይሸጥም ስንል ኦሮሞ ስለሆናችሁ ነው ይሉናል እኛ ዶ አብይ እስካሉ አንናገርም ንብረታችንን መሸጥ ቀርቶ ይረዱን ብልፅግና ከሚደናቀ', '_ ዘመኑ የቴክኖሎጂ ነው በማስረጃ እና በመረጃ ነው አንተ ከዕውቀት ነፃ ስለሆንክ አይፈረድብህም የፈለከውን ጋዜጠኛ አምጣለት በሩ ክፍት ነው', 'የበለጠ መስማማት አልቻልኩም', 'እንኳን ደስ አላችሁ ጤና ሚኒስቴር የአዲስ አበባ ዩኒቨርስቲ ለዶ ር ዋቅጋሪ ዴሬሳ አመንቴ የፕሮፌሰርነት ማዕረግ በመስጠቱ የተሰማውን ደስታ ይገልጻል', 'አቤቱ አምላኬ ሆይ እነ ታጥቦ ጭቃ የሚሰሩትን አያውቁምና ይቅር በላቸው', '_ ምግብ ቤት አስመሰላችሁት እኮ ዝርክርክ', 'በጣም ደደብ ነኝ ይቅርታ', 'ሃሳብ ካለህ በሃሳብህ ሞግት ካላቻልክ ደግሞ ጨዋነትህን ብታሳየን ይመረጣል ስድብ የሰነፍ ሰው ትልቁ ንብረቱ ነው ራስህን ከቻልክ አርም', 'በዓል እንዴት ነው ጥሩ ነው በምን አለፈ በብር', 'ታኬ በጣም በርቱልን ሰዉ ያውራ እናንተ ስሩ ለሀገራቹ ሰርታቹም እያሳያቹን ነው ዶ ር አብይም ኢ ር ታከለ የእኔ ጀግና ናቹ መቼም ታሪክ አይረሳቹም']


In [3]:
for n, i in enumerate(y):
    if i == "1\n":
        y[n] = 1
for n, i in enumerate(y):
    if i == "0\n":
        y[n] = 0
for n, i in enumerate(y):
    if i == "positivene\n":
        y[n] = 0

In [4]:
print(y[0:10])

[1, 1, 1, 0, 0, 1, 1, 0, 0, 0]


In [5]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
y = np.array(y)
label_encoder = LabelEncoder()
vec = label_encoder.fit_transform(y)

In [7]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(vec)
print(y[1])
print(y[3000])

[0. 1.]
[1. 0.]


In [9]:
print(len(y))

3613


In [11]:
import pandas as pd
import numpy as np 
import re,nltk
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state= 42, shuffle = True)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,test_size = 0.20, random_state = 1, shuffle = True)
print(len(X_train))
print(len(X_val))
print(len(X_test))

2312
578
723


In [13]:
tokenizer = Tokenizer(num_words = 30905, filters ='!"#$%&()*+,-./;<=>?@][\\]^{|}~\t\n')
tokenizer.fit_on_texts(X_train)
index_of_words = tokenizer.word_index
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
X_val = tokenizer.texts_to_sequences(X_val)

In [14]:
vocab_size = len(tokenizer.word_index) + 1
maxlen = 100
X_train = pad_sequences(X_train, padding = 'post', maxlen = maxlen)
X_test = pad_sequences(X_test, padding = 'post', maxlen = maxlen)
X_val = pad_sequences (X_val, padding = 'post', maxlen = maxlen)

In [16]:
from numpy import array
from numpy import asarray 
from numpy import zeros
import pickle
with open("EmbeddingAmh.pkl", 'rb') as f:
    embeddings_index = pickle.load(f)
    print('Total %s word vectors.' %len(embeddings_index))

Total 41042 word vectors.


In [17]:
EMBEDDINGS_DIM = 300
embedding_matrix = np.random.random ((len(index_of_words) + 1, EMBEDDINGS_DIM))
for word, i in index_of_words.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
print( 'Embedding matrix shape', embedding_matrix.shape)

Embedding matrix shape (14955, 300)


In [18]:
from keras.layers.convolutional import Conv1D
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dense
from keras.layers import LSTM, SpatialDropout1D, Bidirectional
from keras.layers import GlobalMaxPooling1D
from keras.preprocessing import sequence
import pandas as pd
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from keras import backend as K
model = Sequential()
embedding_layer = Embedding(vocab_size, 300, weights = [embedding_matrix], input_length=maxlen, trainable = False)
model.add(embedding_layer)
model.add(Conv1D(32, kernel_size = 5, activation = 'relu', padding = 'same'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.25))
#model.add(Conv1D(64, kernel_size = 5, activation = 'softmax', padding = 'same'))
#model.add(MaxPooling1D(pool_size =2))
#model.add(Dropout(0.25))

#model.add(Conv1D(64, 5,  activation = 'softmax'))
#model.add(GlobalMaxPooling1D())

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation='sigmoid'))
model.add(Dropout(0.3))
model.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics =['acc'])
model.compile(loss ='categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['acc'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 300)          4486500   
_________________________________________________________________
conv1d (Conv1D)              (None, 100, 32)           48032     
_________________________________________________________________
global_max_pooling1d (Global (None, 32)                0         
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
flatten (Flatten)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 64)                2112      
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 1

In [19]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor = 'val_loss', min_delta = 0.01, patience =4, verbose =1)
callbacks_list = [early_stopping]
hist =model.fit(X_train, y_train, epochs = 50, batch_size = 128, verbose = 1, validation_data = (X_val, y_val))


Epoch 1/50
19/19 [==============================] - 4s 97ms/step - loss: nan - acc: 0.4686 - val_loss: nan - val_acc: 0.4567
Epoch 2/50
19/19 [==============================] - 1s 60ms/step - loss: nan - acc: 0.4557 - val_loss: nan - val_acc: 0.4567
Epoch 3/50
19/19 [==============================] - 1s 52ms/step - loss: nan - acc: 0.4808 - val_loss: nan - val_acc: 0.4567
Epoch 4/50
19/19 [==============================] - 1s 53ms/step - loss: nan - acc: 0.4767 - val_loss: nan - val_acc: 0.4567
Epoch 5/50
19/19 [==============================] - 1s 59ms/step - loss: nan - acc: 0.4582 - val_loss: nan - val_acc: 0.4567
Epoch 6/50
19/19 [==============================] - 1s 65ms/step - loss: nan - acc: 0.4880 - val_loss: nan - val_acc: 0.4567
Epoch 7/50
19/19 [==============================] - 1s 56ms/step - loss: nan - acc: 0.4592 - val_loss: nan - val_acc: 0.4567
Epoch 8/50
19/19 [==============================] - 1s 67ms/step - loss: nan - acc: 0.4623 - val_loss: nan - val_acc: 0.4567


In [20]:
score,acc  = model.evaluate(X_test, y_test, verbose=1)
print('Test accuracy:', acc)

23/23 [==============================] - 0s 8ms/step - loss: nan - acc: 0.4592
Test accuracy: 0.4591977894306183


In [21]:
_,train_acc = model.evaluate(X_train,y_train, verbose = 0)
print('Train accuracy:', train_acc)

Train accuracy: 0.4714532792568207


In [22]:
a, b = model.evaluate(X_val, y_val, verbose =0)
print(b)

0.4567474126815796
